In [ ]:
import math
import numpy as np
import pandas as pd
from pandas import Series
#
import matplotlib.pyplot as plt
#
import scipy
from scipy import stats
from scipy.stats import entropy
from scipy.special import rel_entr, kl_div
#
# import matplotlib.gridspec as gridspec
import seaborn as sns
#
from sklearn.metrics import mean_squared_error

In [ ]:
scenario_name_set = ["uni-01", "uni-02", "uni-03", "uni-04", "uni-05", "uni-06", "uni-07", "uni-08", "uni-09"]
#
fps_values = [25]
r_contact = 2.0
fps = fps_values[0]
dt = 1.0/fps
#
scenario_name = scenario_name_set[0]
readfile_name = "traj-classification-features_"+scenario_name+".txt"
print ("open", readfile_name)
#
record_raw = pd.read_csv(readfile_name, sep= ",", header=0) 
#
# iterating the columns
for col in record_raw.columns:
    print(col)
#

In [ ]:
def generate_histograms(scenario_name_set, fps, variable_x, label_x, x_binwidth, x_min_i="", x_max_i="", xtick_space=""):
    #
    # prepare subplots
    #
    n_types = 1
    n_columns = n_types
    n_rows = len(scenario_name_set)
    subplot_x = 4.0
    subplot_y = 3.0
    figsize_x = subplot_x*n_columns
    figsize_y = subplot_y*n_rows
    fig = plt.figure(figsize=(figsize_x, figsize_y))       
    #
    check_x_min = False
    check_x_max = False
    if not x_min_i:
        check_x_min = True
    if not x_max_i:
        check_x_max = True
    #
    for i in range (0, len(scenario_name_set)):
        dt = 1.0/fps
        scenario_name = scenario_name_set[i]
        readfile_name = "traj-classification-features_"+scenario_name+".txt"
        record_raw = pd.read_csv(readfile_name, sep= ",", header=0) 
        #
        x_0 = record_raw[variable_x] # all
        x = [x_0]
        #
        subplot_title = []
        subplot_title.append(scenario_name_set[i])
        #
        if (check_x_min == True):
            x_min = min(min(x[0]), 0.0)  
        else: 
            x_min = x_min_i
        if (check_x_max == True):
            x_max = max(x[0])  
        else: 
            x_max = x_max_i            
        #
        n_xticks = 5
        #
        for j in range (0, n_columns):
            ax = fig.add_subplot(n_rows,n_columns,(n_columns*i+j+1))
            #
            bin_list = np.arange(x_min, x_max*1.05, x_binwidth) 
            # counts = plt.hist(x[j], bins=bin_list, alpha=0.5, weights=np.ones_like(x[j])/len(x[j])) # alpha controls transparency
            counts = plt.hist(x[j], bins=bin_list, alpha=0.5) # alpha controls transparency
            #
            plt.xlim(x_min, x_max)
            if not xtick_space:
                ax.xaxis.set_major_locator(plt.MaxNLocator(n_xticks))
            else:
                plt.xticks(np.arange(x_min, x_max*1.05, xtick_space))       
            plt.tick_params(labelsize=12)
            if (i == (len(scenario_name_set)-1)):
                plt.xlabel(label_x, fontsize=12)
            if (j == 0):
                plt.ylabel('frequency', fontsize=12) 
            plt.title(subplot_title[j], fontsize=16)               
    # set the spacing between subplots
    plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.4, hspace=0.4)    
    plt.show()        

In [ ]:
def generate_Wasserstein_v(scenario_name_set, fps, x_binwidth, x_min_i="", x_max_i="", xtick_space=""):
    #
    variable_x1 = "v_avg"
    variable_x2 = "v_ini"
    label_x = "contact duration (s)"
    x_min = 0.0
    #
    # prepare subplots
    #
    n_types = 1
    n_columns = n_types
    n_rows = len(scenario_name_set)
    subplot_x = 4.0
    subplot_y = 3.0
    figsize_x = subplot_x*n_columns
    figsize_y = subplot_y*n_rows
    fig = plt.figure(figsize=(figsize_x, figsize_y))       
    #
    check_x_max = False
    if not x_max_i:
        check_x_max = True
    #
    print_header = "scenario\tWasserstein\t%_ballistic"
    print(print_header)    
    #
    for i in range (0, len(scenario_name_set)):
        dt = 1.0/fps
        scenario_name = scenario_name_set[i]
        readfile_name = "traj-classification-features_"+scenario_name+".txt"
        record_raw = pd.read_csv(readfile_name, sep= ",", header=0) 
        #
        x_a0 = record_raw[variable_x1] # all
        x_b0 = record_raw[variable_x2] # all
        x_a = [x_a0]
        x_b = [x_b0]
        count_ballistic = len(record_raw[variable_x1][(record_raw['motion_type']=="B")]) # ballistic motion)
        #
        subplot_title = []
        subplot_title.append(scenario_name_set[i])
        #
        if (check_x_max == True):
            x_max = max(x[0])  
        else: 
            x_max = x_max_i            
        #
        n_xticks = 5
        #
        print_wasserstein = scenario_name_set[i]
        for j in range (0, n_columns):
            ax = fig.add_subplot(n_rows,n_columns,(n_columns*i+j+1))
            #
            bin_list = np.arange(x_min, x_max*1.05, x_binwidth) 
            counts1 = plt.hist(x_a[j], bins=bin_list, weights=np.ones_like(x_a[j])/len(x_a[j]), alpha=0.5) # alpha controls transparency
            counts2 = plt.hist(x_b[j], bins=bin_list, weights=np.ones_like(x_b[j])/len(x_b[j]), histtype="step", linewidth=3)
            #
            wasserstein_value = stats.wasserstein_distance(counts1[0], counts2[0])
            #
            plt.xlim(x_min, x_max)
            if not xtick_space:
                ax.xaxis.set_major_locator(plt.MaxNLocator(n_xticks))
            else:
                plt.xticks(np.arange(x_min, x_max*1.05, xtick_space))       
            plt.tick_params(labelsize=12)
            if (i == (len(scenario_name_set)-1)):
                plt.xlabel(label_x, fontsize=12)
            if (j == 0):
                plt.ylabel('relative frequency', fontsize=12) 
                #
                print_wasserstein += "\t{:.4f}".format(wasserstein_value)+"\t{:.2f}".format(count_ballistic/len(x_a0)*100.0)
                print(print_wasserstein) 
            plt.title(subplot_title[j], fontsize=16)   
    # set the spacing between subplots
    plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.4, hspace=0.4)    
    plt.show()        

In [ ]:
def generate_histograms_relative(scenario_name_set, fps, variable_x, label_x, x_binwidth, x_min_i="", x_max_i="", xtick_space=""):
    #
    # prepare subplots
    #
    n_types = 1
    n_columns = n_types
    n_rows = len(scenario_name_set)
    subplot_x = 4.0
    subplot_y = 3.0
    figsize_x = subplot_x*n_columns
    figsize_y = subplot_y*n_rows
    fig = plt.figure(figsize=(figsize_x, figsize_y))       
    #
    check_x_min = False
    check_x_max = False
    if not x_min_i:
        check_x_min = True
    if not x_max_i:
        check_x_max = True
    #
    for i in range (0, len(scenario_name_set)):
        dt = 1.0/fps
        scenario_name = scenario_name_set[i]
        readfile_name = "traj-classification-features_"+scenario_name+".txt"
        record_raw = pd.read_csv(readfile_name, sep= ",", header=0) 
        #
        x_0 = record_raw[variable_x] # all
        x = [x_0]
        count_ballistic = len(record_raw[variable_x1][(record_raw['motion_type']=="B")]) # ballistic motion)
        #
        subplot_title = []
        subplot_title.append(scenario_name_set[i])
        #
        if (check_x_min == True):
            x_min = min(min(x[0]), 0.0)  
        else: 
            x_min = x_min_i
        if (check_x_max == True):
            x_max = max(x[0])  
        else: 
            x_max = x_max_i            
        #
        n_xticks = 5
        #
        for j in range (0, n_columns):
            ax = fig.add_subplot(n_rows,n_columns,(n_columns*i+j+1))
            #
            bin_list = np.arange(x_min, x_max*1.05, x_binwidth) 
            counts = plt.hist(x[j], bins=bin_list, alpha=0.5, weights=np.ones_like(x[j])/len(x[j])) # alpha controls transparency
            #
            plt.xlim(x_min, x_max)
            if not xtick_space:
                ax.xaxis.set_major_locator(plt.MaxNLocator(n_xticks))
            else:
                plt.xticks(np.arange(x_min, x_max*1.05, xtick_space))       
            plt.tick_params(labelsize=12)
            if (i == (len(scenario_name_set)-1)):
                plt.xlabel(label_x, fontsize=12)
            if (j == 0):
                plt.ylabel('relative frequency', fontsize=12) 
            plt.title(subplot_title[j], fontsize=16)               
    # set the spacing between subplots
    plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.4, hspace=0.4)    
    plt.show()        

In [ ]:
def generate_histograms_deg(scenario_name_set, fps, variable_x, label_x, x_binwidth, x_min_i = "", x_max_i = ""):
    #
    # prepare subplots
    #
    n_types = 1
    n_columns = n_types
    n_rows = len(scenario_name_set)
    subplot_x = 4.0
    subplot_y = 3.0
    figsize_x = subplot_x*n_columns
    figsize_y = subplot_y*n_rows
    fig = plt.figure(figsize=(figsize_x, figsize_y))       
    #
    check_x_min = False
    check_x_max = False
    if not x_min_i:
        check_x_min = True
    if not x_max_i:
        check_x_max = True           
    #
    for i in range (0, len(scenario_name_set)):
        dt = 1.0/fps
        scenario_name = scenario_name_set[i]
        readfile_name = "traj-classification-features_"+scenario_name+".txt"
        record_raw = pd.read_csv(readfile_name, sep= ",", header=0) 
        #
        x_0 = record_raw[variable_x] # all
        x = [x_0/np.pi*180] # deg
        #
        subplot_title = []
        subplot_title.append(scenario_name_set[i])
        #
        if (check_x_min == True):
            x_min = min(min(x[0]), 0.0)  
        else: 
            x_min = x_min_i
        if (check_x_max == True):
            x_max = max(x[0])  
        else: 
            x_max = x_max_i    
        #
        xtick_space = 45
        if (variable_x == "alpha_std"):
            xtick_space = 30  
        #
        for j in range (0, n_columns):
            ax = fig.add_subplot(n_rows,n_columns,(n_columns*i+j+1))
            #
            bin_list = np.arange(x_min, x_max*1.05, x_binwidth) # x_min, x_max, x_binwidth are given in deg
            # counts = plt.hist(x[j], bins=bin_list, alpha=0.5, weights=np.ones_like(x[j])/len(x[j])) # alpha controls transparency
            counts = plt.hist(x[j], bins=bin_list, alpha=0.5) # alpha controls transparency
            #
            plt.xticks(np.arange(x_min, x_max*1.05, xtick_space))
            plt.xlim(x_min, x_max)              
            plt.tick_params(labelsize=12)
            if (i == (len(scenario_name_set)-1)):
                plt.xlabel(label_x, fontsize=12)
            if (j == 0):
                plt.ylabel('frequency', fontsize=12) 
            plt.title(subplot_title[j], fontsize=16)               
    # set the spacing between subplots
    plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.4, hspace=0.4)    
    plt.show()

In [ ]:
def generate_histograms_relative_deg(scenario_name_set, fps, variable_x, label_x, x_binwidth, x_min_i = "", x_max_i = ""):
    #
    # prepare subplots
    #
    n_types = 1
    n_columns = n_types
    n_rows = len(scenario_name_set)
    subplot_x = 4.0
    subplot_y = 3.0
    figsize_x = subplot_x*n_columns
    figsize_y = subplot_y*n_rows
    fig = plt.figure(figsize=(figsize_x, figsize_y))       
    #
    check_x_min = False
    check_x_max = False
    if not x_min_i:
        check_x_min = True
    if not x_max_i:
        check_x_max = True           
    #
    for i in range (0, len(scenario_name_set)):
        dt = 1.0/fps
        scenario_name = scenario_name_set[i]
        readfile_name = "traj-classification-features_"+scenario_name+".txt"
        record_raw = pd.read_csv(readfile_name, sep= ",", header=0) 
        #
        x_0 = record_raw[variable_x] # all
        x = [x_0/np.pi*180] # deg
        #
        subplot_title = []
        subplot_title.append(scenario_name_set[i])
        #
        if (check_x_min == True):
            x_min = min(min(x[0]), 0.0)  
        else: 
            x_min = x_min_i
        if (check_x_max == True):
            x_max = max(x[0])  
        else: 
            x_max = x_max_i    
        #
        xtick_space = 45
        if (variable_x == "alpha_std"):
            xtick_space = 30  
        #
        for j in range (0, n_columns):
            ax = fig.add_subplot(n_rows,n_columns,(n_columns*i+j+1))
            #
            bin_list = np.arange(x_min, x_max*1.05, x_binwidth) # x_min, x_max, x_binwidth are given in deg
            counts = plt.hist(x[j], bins=bin_list, alpha=0.5, weights=np.ones_like(x[j])/len(x[j])) # alpha controls transparency
            #
            plt.xticks(np.arange(x_min, x_max*1.05, xtick_space))
            plt.xlim(x_min, x_max)              
            plt.tick_params(labelsize=12)
            if (i == (len(scenario_name_set)-1)):
                plt.xlabel(label_x, fontsize=12)
            if (j == 0):
                plt.ylabel('relative frequency', fontsize=12) 
            plt.title(subplot_title[j], fontsize=16)               
    # set the spacing between subplots
    plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.4, hspace=0.4)    
    plt.show()

In [ ]:
def generate_scatterplot(scenario_name_set, fps, 
                         variable_x, variable_y, label_x, label_y,
                         x_min_i="", x_max_i="", xtick_space="", 
                         y_min_i="", y_max_i="", ytick_space=""):
    #
    # prepare subplots
    #
    n_types = 1
    n_columns = n_types
    n_rows = len(scenario_name_set)
    subplot_x = 4.0
    subplot_y = 3.5
    figsize_x = subplot_x*n_columns
    figsize_y = subplot_y*n_rows
    fig = plt.figure(figsize=(figsize_x, figsize_y))       
    #
    check_x_min = False
    check_x_max = False
    if not x_min_i:
        check_x_min = True
    if not x_max_i:
        check_x_max = True
    #
    check_y_min = False
    check_y_max = False
    if not y_min_i:
        check_y_min = True
    if not y_max_i:
        check_y_max = True        
    #
    for i in range (0, len(scenario_name_set)):
        dt = 1.0/fps
        scenario_name = scenario_name_set[i]
        readfile_name = "traj-classification-features_"+scenario_name+".txt"
        record_raw = pd.read_csv(readfile_name, sep= ",", header=0)     
        #
        # x axis
        #
        x_0 = record_raw[variable_x] # all
        x = [x_0]
        #
        if (variable_x in ["alpha_avg", "turning_angle_avg"]):
            x = [x_0/np.pi*180] # deg                 
        #
        # y axis
        #
        y_0 = record_raw[variable_y] # all 
        y = [y_0]   
        #
        if (variable_y in ["alpha_avg", "turning_angle_avg"]):
            y = [y_0/np.pi*180] # deg               
        #
        subplot_title = []
        subplot_title.append(scenario_name_set[i])
        #
        if (check_x_min == True):
            x_min = min(min(x[0]), 0.0)  
        else: 
            x_min = x_min_i
        if (check_x_max == True):
            x_max = max(x[0])  
        else: 
            x_max = x_max_i  
        #
        if (check_y_min == True):
            y_min = min(min(y[0]), 0.0)  
        else: 
            y_min = y_min_i
        if (check_y_max == True):
            y_max = max(y[0])  
        else: 
            y_max = y_max_i  
        #        
        n_xticks = 4
        n_yticks = 4
        #
        for j in range (0, n_columns):
            ax = fig.add_subplot(n_rows,n_columns,(n_columns*i+j+1))
            plt.scatter(x[j], y[j], alpha=0.1, s=2)
            #
            plt.xlim(x_min, x_max)
            if not xtick_space:
                ax.xaxis.set_major_locator(plt.MaxNLocator(n_xticks))
            else:
                plt.xticks(np.arange(x_min, x_max*1.05, xtick_space))               
            #
            plt.ylim(y_min, y_max)
            if not ytick_space:
                ax.yaxis.set_major_locator(plt.MaxNLocator(n_yticks))
            else:
                plt.yticks(np.arange(y_min, y_max*1.05, ytick_space))                  
            #
            plt.tick_params(labelsize=12)
            if (i == (len(scenario_name_set)-1)):
                plt.xlabel(label_x, fontsize=12)
            if (j == 0):
                plt.ylabel(label_y, fontsize=12)
            plt.title(subplot_title[j], fontsize=16)
    # set the spacing between subplots
    plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.4, hspace=0.4)    
    plt.show()

In [ ]:
def generate_scatterplot_trend(scenario_name_set, fps, 
                         variable_x, variable_y, label_x, label_y, x_binwidth,
                         x_min_i="", x_max_i="", xtick_space="", 
                         y_min_i="", y_max_i="", ytick_space=""):
    #
    # prepare subplots
    #
    n_types = 1
    n_columns = n_types
    n_rows = len(scenario_name_set)
    subplot_x = 4.0
    subplot_y = 3.5
    figsize_x = subplot_x*n_columns
    figsize_y = subplot_y*n_rows
    fig = plt.figure(figsize=(figsize_x, figsize_y))       
    #
    check_x_min = False
    check_x_max = False
    if not x_min_i:
        check_x_min = True
    if not x_max_i:
        check_x_max = True
    #
    check_y_min = False
    check_y_max = False
    if not y_min_i:
        check_y_min = True
    if not y_max_i:
        check_y_max = True        
    #
    for i in range (0, len(scenario_name_set)):
        dt = 1.0/fps
        scenario_name = scenario_name_set[i]
        readfile_name = "traj-classification-features_"+scenario_name+".txt"
        record_raw = pd.read_csv(readfile_name, sep= ",", header=0)     
        #
        # x axis
        #
        x_0 = record_raw[variable_x] # all
        x = [x_0]
        #
        if (variable_x in ["alpha_avg", "turning_angle_avg"]):
            x = [x_0/np.pi*180] # deg         
        #
        # y axis
        #
        y_0 = record_raw[variable_y] # all        
        y = [y_0]       
        #
        #
        if (variable_y in ["alpha_avg", "turning_angle_avg"]):
            y = [y_0/np.pi*180] # deg         
        #
        subplot_title = []
        subplot_title.append(scenario_name_set[i])
        #
        if (check_x_min == True):
            x_min = min(min(x[0]), 0.0)  
        else: 
            x_min = x_min_i
        if (check_x_min == True):
            x_max = max(x[0])  
        else: 
            x_max = x_max_i  
        #
        if (check_y_min == True):
            y_min = min(min(y[0]), 0.0)  
        else: 
            y_min = y_min_i
        if (check_y_max == True):
            y_max = max(y[0])  
        else: 
            y_max = y_max_i  
        #        
        n_xticks = 4
        n_yticks = 4
        #
        for j in range (0, n_columns):
            #
            ax = fig.add_subplot(n_rows,n_columns,(n_columns*i+j+1))
            plt.scatter(x[j], y[j], alpha=0.1, s=2)
            #
            binlist = np.arange(0, x_max, x_binwidth) 
            bin_means, bin_edges, bin_number = stats.binned_statistic(x[j], y[j], statistic='mean', bins=binlist)
            bin_count, bin_edges, bin_number = stats.binned_statistic(x[j], y[j], statistic='count', bins=binlist)
            # bin_std, bin_edges, bin_number = stats.binned_statistic(x[j], y[j], statistic='std', bins=len(binlist))
            bin_width = (bin_edges[1] - bin_edges[0])
            bin_centers = bin_edges[1:] - bin_width/2
            plt.plot(bin_centers, bin_means, color = 'r', lw=3)  
            #
            plt.xlim(x_min, x_max)
            if not xtick_space:
                ax.xaxis.set_major_locator(plt.MaxNLocator(n_xticks))
            else:
                plt.xticks(np.arange(x_min, x_max*1.05, xtick_space))               
            #
            plt.ylim(y_min, y_max)
            if not ytick_space:
                ax.yaxis.set_major_locator(plt.MaxNLocator(n_yticks))
            else:
                plt.yticks(np.arange(y_min, y_max*1.05, ytick_space))                  
            #
            plt.tick_params(labelsize=12)
            if (i == (len(scenario_name_set)-1)):
                plt.xlabel(label_x, fontsize=12)
            if (j == 0):
                plt.ylabel(label_y, fontsize=12)
            plt.title(subplot_title[j], fontsize=16)
    # set the spacing between subplots
    plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.4, hspace=0.4)    
    plt.show()

In [ ]:
def generate_heatmap(scenario_name_set, fps, 
                         variable_x, variable_y, label_x, label_y,
                         x_min_i="", x_max_i="", xtick_space="", 
                         y_min_i="", y_max_i="", ytick_space=""):
    #
    # prepare subplots
    #
    n_types = 1
    n_columns = n_types
    n_rows = len(scenario_name_set)
    subplot_x = 4.0
    subplot_y = 3.5
    figsize_x = subplot_x*n_columns
    figsize_y = subplot_y*n_rows
    fig = plt.figure(figsize=(figsize_x, figsize_y))       
    #
    check_x_min = False
    check_x_max = False
    if not x_min_i:
        check_x_min = True
    if not x_max_i:
        check_x_max = True
    #
    check_y_min = False
    check_y_max = False
    if not y_min_i:
        check_y_min = True
    if not y_max_i:
        check_y_max = True        
    #
    for i in range (0, len(scenario_name_set)):
        dt = 1.0/fps
        scenario_name = scenario_name_set[i]
        readfile_name = "traj-classification-features_"+scenario_name+".txt"
        record_raw = pd.read_csv(readfile_name, sep= ",", header=0) 
        #
        # x axis
        #
        x_0 = record_raw[variable_x] # all
        x = [x_0]
        #
        if (variable_x == "n_neighbors_avg"):
            x = [x_0/(r_contact*r_contact*np.pi)]
        if (variable_x == "n_neighbors_std"):
            x = [x_0/(r_contact*r_contact*np.pi)]   
        if (variable_x == "frame_start"):
            x = [x_0/fps]
        if (variable_x == "frame_end"):
            x = [x_0/fps]   
        if (variable_x == "alpha_avg"):
            x = [x_0/np.pi*180] # deg
        if (variable_x == "turning_angle_avg"):
            x = [x_0/np.pi*180] # deg              
        #
        # y axis
        #
        y_0 = record_raw[variable_y] # all       
        y = [y_0]       
        #
        if (variable_y == "alpha_avg"):
            y = [y_0/np.pi*180, y_1/np.pi*180, y_2/np.pi*1800] # deg  
        if (variable_y == "turning_angle_avg"):
            y = [y_0/np.pi*180, y_1/np.pi*180, y_2/np.pi*1800] # deg             
        #
        subplot_title = []
        subplot_title.append(scenario_name_set[i]+" (all)")
        subplot_title.append("(parallel)")
        subplot_title.append("(crossing)")   
        #
        if (check_x_min == True):
            x_min = min(min(x[0]), 0.0)  
        else: 
            x_min = x_min_i
        if (check_x_max == True):
            x_max = max(x[0])  
        else: 
            x_max = x_max_i  
        #
        if (check_y_min == True):
            y_min = min(min(y[0]), 0.0)  
        else: 
            y_min = y_min_i
        if (check_y_max == True):
            y_max = max(y[0])  
        else: 
            y_max = y_max_i  
        #        
        n_xticks = 4
        n_yticks = 4
        #
        # print(type(x[1]), type(y[1]))
        for j in range (0, n_columns):
            #
            x_series = pd.Series(x[j])
            y_series = pd.Series(y[j])
            df_input = pd.DataFrame({'x': x_series,'y': y_series})    
            df_input = df_input.dropna()
            #
            ax = fig.add_subplot(n_rows,n_columns,(n_columns*i+j+1))
            plt.hist2d(df_input['x'], df_input['y'], bins=(20, 20), range=[[x_min, x_max*1.05],[y_min, y_max*1.05]], cmap=plt.cm.Reds)          
            #
            plt.xlim(x_min, x_max)
            if not xtick_space:
                ax.xaxis.set_major_locator(plt.MaxNLocator(n_xticks))
            else:
                plt.xticks(np.arange(x_min, x_max*1.05, xtick_space))               
            #
            plt.ylim(y_min, y_max)
            if not ytick_space:
                ax.yaxis.set_major_locator(plt.MaxNLocator(n_yticks))
            else:
                plt.yticks(np.arange(y_min, y_max*1.05, ytick_space))                  
            #
            plt.tick_params(labelsize=12)
            if (i == (len(scenario_name_set)-1)):
                plt.xlabel(label_x, fontsize=12)
            if (j == 0):
                plt.ylabel(label_y, fontsize=12)
            plt.title(subplot_title[j], fontsize=16)
            plt.colorbar()
    # set the spacing between subplots
    plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.2, hspace=0.4)    
    #   
    plt.show()

In [ ]:
def generate_histograms_classification0(scenario_name_set, fps, variable_x, label_x, x_binwidth, x_min_i="", x_max_i="", xtick_space=""):
    #
    # prepare subplots
    #
    n_types = 1
    n_classifications = 3
    n_scenarios = len(scenario_name_set)
    #
    n_columns = n_classifications+1
    n_rows    = n_types*n_scenarios
    #
    subplot_x = 4.0
    subplot_y = 3.0
    figsize_x = subplot_x*n_columns
    figsize_y = subplot_y*n_rows
    fig = plt.figure(figsize=(figsize_x, figsize_y))       
    #
    check_x_min = False
    check_x_max = False
    if not x_min_i:
        check_x_min = True
    if not x_max_i:
        check_x_max = True
    #
    # for i in range (0, len(scenario_name_set)):
    for scenario_index in range(0, len(scenario_name_set)):
        scenario_name = scenario_name_set[scenario_index]
        for i in range(0, n_types):
            row_id = scenario_index*n_types+i
            dt = 1.0/fps
            # scenario_name = scenario_name_set[i]
            readfile_name = "traj-classification-features_"+scenario_name+".txt"
            record_raw = pd.read_csv(readfile_name, sep= ",", header=0) 
            #
            x_0 = record_raw[variable_x] # all
            subplot_title_0 = scenario_name
            #
            x_1 = x_0[(record_raw['motion_type']=="B")] # ballistic motion
            x_2 = x_0[(record_raw['motion_type']=="C")] # confined motion
            x_3 = x_0[(record_raw['motion_type']=="S")] # sub-ballistic motion
            x = [x_0, x_1, x_2, x_3]
            #
            subplot_title = []
            subplot_title.append(subplot_title_0)
            subplot_title.append("(ballistic)")
            subplot_title.append("(confined motion)")
            subplot_title.append("(sub-ballistic)")
            #
            if (check_x_min == True):
                x_min = min(min(x[0]), 0.0)  
            else: 
                x_min = x_min_i
            if (check_x_max == True):
                x_max = max(x[0])  
            else: 
                x_max = x_max_i            
            #
            n_xticks = 5
            #
            for j in range(0, n_columns):
                if (len(x[j]) > 0):
                    ax = fig.add_subplot(n_rows,n_columns,(n_columns*row_id+j+1))
                    #
                    bin_list = np.arange(x_min, x_max*1.05, x_binwidth) 
                    # counts = plt.hist(x[j], bins=bin_list, alpha=0.5, weights=np.ones_like(x[j])/len(x[j])) # alpha controls transparency
                    counts = plt.hist(x[j], bins=bin_list, alpha=0.5) # alpha controls transparency
                    #
                    plt.xlim(x_min, x_max)
                    if not xtick_space:
                        ax.xaxis.set_major_locator(plt.MaxNLocator(n_xticks))
                    else:
                        plt.xticks(np.arange(x_min, x_max*1.05, xtick_space))       
                    plt.tick_params(labelsize=12)
                    if (row_id == (n_rows-1)):
                        plt.xlabel(label_x, fontsize=12)
                    if (j == 0):
                        plt.ylabel('frequency', fontsize=12) 
                    plt.title(subplot_title[j], fontsize=16)               
    # set the spacing between subplots
    plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.4, hspace=0.4)    
    plt.show()        

In [ ]:
def generate_histograms_classification(scenario_name_set, fps, variable_x, label_x, x_binwidth, x_min_i="", x_max_i="", xtick_space=""):
    #
    # prepare subplots
    #
    n_types = 1
    n_classifications = 3
    n_scenarios = len(scenario_name_set)
    #
    n_columns = n_classifications+1
    n_rows    = n_types*n_scenarios
    #
    subplot_x = 4.0
    subplot_y = 3.0
    figsize_x = subplot_x*n_columns
    figsize_y = subplot_y*n_rows
    fig = plt.figure(figsize=(figsize_x, figsize_y))       
    #
    check_x_min = False
    check_x_max = False
    if not x_min_i:
        check_x_min = True
    if not x_max_i:
        check_x_max = True
    #
    # for i in range (0, len(scenario_name_set)):
    for scenario_index in range(0, len(scenario_name_set)):
        scenario_name = scenario_name_set[scenario_index]
        for i in range(0, n_types):
            row_id = scenario_index*n_types+i
            dt = 1.0/fps
            # scenario_name = scenario_name_set[i]
            readfile_name = "traj-classification-features_"+scenario_name+".txt"
            record_raw = pd.read_csv(readfile_name, sep= ",", header=0) 
            #
            x_0 = record_raw[variable_x] # all
            subplot_title_0 = scenario_name
            subplot_title_1 = ""
            #
            x_1 = x_0[(record_raw['motion_type']=="B")] # ballistic motion
            x_2 = x_0[(record_raw['motion_type']=="C")] # confined motion
            x_3 = x_0[(record_raw['motion_type']=="S")] # sub-ballistic motion
            x = [x_0, x_1, x_2, x_3]
            #
            subplot_title = []
            subplot_title.append(subplot_title_0)
            if (i == 0):
                subplot_title.append(subplot_title_1+"ballistic")
                subplot_title.append(subplot_title_1+"confined")
                subplot_title.append(subplot_title_1+"sub-ballistic")
            else:
                subplot_title.append(subplot_title_0+" & ballistic")
                subplot_title.append(subplot_title_0+" & confined")
                subplot_title.append(subplot_title_0+" & sub-ballistic")  
            #
            if (check_x_min == True):
                x_min = min(min(x[0]), 0.0)  
            else: 
                x_min = x_min_i
            if (check_x_max == True):
                x_max = max(x[0])  
            else: 
                x_max = x_max_i            
            #
            n_xticks = 5            
            #
            for j in range(0, n_columns):
                if (len(x[j]) > 0):
                    ax = fig.add_subplot(n_rows,n_columns,(n_columns*row_id+j+1))
                    #
                    bin_list = np.arange(x_min, x_max*1.05, x_binwidth) 
                    # counts = plt.hist(x[j], bins=bin_list, alpha=0.5, weights=np.ones_like(x[j])/len(x[j])) # alpha controls transparency
                    counts = plt.hist(x[j], bins=bin_list, alpha=0.5) # alpha controls transparency
                    #
                    plt.xlim(x_min, x_max)
                    if not xtick_space:
                        ax.xaxis.set_major_locator(plt.MaxNLocator(n_xticks))
                    else:
                        plt.xticks(np.arange(x_min, x_max*1.05, xtick_space))       
                    plt.tick_params(labelsize=14)
                    if (row_id == (n_rows-1)):
                        plt.xlabel(label_x, fontsize=16)
                    if (j == 0):
                        plt.ylabel('frequency', fontsize=16) 
                    plt.title(subplot_title[j], fontsize=18)                
    # set the spacing between subplots
    plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.4, hspace=0.4)    
    plt.show() 


In [ ]:
def generate_histograms_classification_relative(scenario_name_set, fps, variable_x, label_x, x_binwidth, x_min_i="", x_max_i="", xtick_space=""):
    #
    # prepare subplots
    #
    n_types = 1
    n_classifications = 3
    n_scenarios = len(scenario_name_set)
    #
    n_columns = n_classifications+1
    n_rows    = n_types*n_scenarios
    #
    subplot_x = 4.0
    subplot_y = 3.0
    figsize_x = subplot_x*n_columns
    figsize_y = subplot_y*n_rows
    fig = plt.figure(figsize=(figsize_x, figsize_y))       
    #
    check_x_min = False
    check_x_max = False
    if not x_min_i:
        check_x_min = True
    if not x_max_i:
        check_x_max = True
    #
    # for i in range (0, len(scenario_name_set)):
    for scenario_index in range(0, len(scenario_name_set)):
        scenario_name = scenario_name_set[scenario_index]
        for i in range(0, n_types):
            row_id = scenario_index*n_types+i
            dt = 1.0/fps
            # scenario_name = scenario_name_set[i]
            readfile_name = "traj-classification-features_"+scenario_name+".txt"
            record_raw = pd.read_csv(readfile_name, sep= ",", header=0) 
            #
            x_0 = record_raw[variable_x] # all
            subplot_title_0 = scenario_name
            #
            x_1 = x_0[(record_raw['motion_type']=="B")] # ballistic motion
            x_2 = x_0[(record_raw['motion_type']=="C")] # confined motion
            x_3 = x_0[(record_raw['motion_type']=="S")] # sub-ballistic motion
            x = [x_0, x_1, x_2, x_3]
            #
            subplot_title = []
            subplot_title.append(subplot_title_0)
            subplot_title.append("(ballistic)")
            subplot_title.append("(confined motion)")
            subplot_title.append("(sub-ballistic)")
            #
            if (check_x_min == True):
                x_min = min(min(x[0]), 0.0)  
            else: 
                x_min = x_min_i
            if (check_x_max == True):
                x_max = max(x[0])  
            else: 
                x_max = x_max_i            
            #
            n_xticks = 5
            #
            for j in range(0, n_columns):
                if (len(x[j]) > 0):
                    ax = fig.add_subplot(n_rows,n_columns,(n_columns*row_id+j+1))
                    #
                    bin_list = np.arange(x_min, x_max*1.05, x_binwidth) 
                    counts = plt.hist(x[j], bins=bin_list, alpha=0.5, weights=np.ones_like(x[j])/len(x[j])) # alpha controls transparency
                    #
                    plt.xlim(x_min, x_max)
                    if not xtick_space:
                        ax.xaxis.set_major_locator(plt.MaxNLocator(n_xticks))
                    else:
                        plt.xticks(np.arange(x_min, x_max*1.05, xtick_space))       
                    plt.tick_params(labelsize=12)
                    if (row_id == (n_rows-1)):
                        plt.xlabel(label_x, fontsize=12)
                    if (j == 0):
                        plt.ylabel('relative frequency', fontsize=12) 
                    plt.title(subplot_title[j], fontsize=16)               
    # set the spacing between subplots
    plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.4, hspace=0.4)    
    plt.show()        

In [ ]:
def show_classification_count(scenario_name_set, variable_x):
    #
    print_header = "\tNo_contacts\t\t\t"
    print_header += "\nscenario\tcount_all\tall_ballistic\tall_confined\tall_sub-ballistic"
    # print_header += "\tcount_parallel\tparallel_ballistic\tparallel_confined\tparallel_sub-ballistic"
    # print_header += "\tcount_head-on\thead-on_ballistic\thead-on_confined\thead-on_sub-ballistic"
    print(print_header)
    #     
    for row_id in range(0, len(scenario_name_set)):
        scenario_name = scenario_name_set[row_id]
        readfile_name = "traj-classification-features_"+scenario_name+".txt"
        record_raw = pd.read_csv(readfile_name, sep= ",", header=0) 
        print_count = scenario_name
        print_ratio = ""
        #
        x_all  = record_raw[variable_x] # all
        x_base = [x_all]
        #
        n_classifications = 3
        for j in range(0, len(x_base)):
            count_ballistic    = len(x_base[j][(record_raw['motion_type']=="B")]) # ballistic motion
            count_confined     = len(x_base[j][(record_raw['motion_type']=="C")]) # confined motion
            count_subballistic = len(x_base[j][(record_raw['motion_type']=="S")]) # sub-ballistic motion
            count_subtotal     = count_ballistic+count_confined+count_subballistic
            print_count += "\t"+str(count_subtotal)+"\t"+str(count_ballistic)+"\t"+str(count_confined)+"\t"+str(count_subballistic)
            print_ratio += "\t"
            print_ratio += "\t{:.2f}".format(count_ballistic/count_subtotal*100)
            print_ratio += "\t{:.2f}".format(count_confined/count_subtotal*100)
            print_ratio += "\t{:.2f}".format(count_subballistic/count_subtotal*100)                
        #
        print(print_count)
        print(print_ratio)

In [ ]:
def show_classification_summary1(scenario_name_set, variable_x):
    #
    print_header = "scenario\tcategory\t_all\tall_ballistic\tall_confined\tall_subballistic"
    # print_header += "\t_parallel\tparallel_ballistic\tparallel_confined\tparallel_subballistic"
    # print_header += "\t_head-on\thead-on_ballistic\thead-on_confined\thead-on_subballistic"
    print(print_header)
    #     
    for row_id in range(0, len(scenario_name_set)):
        scenario_name = scenario_name_set[row_id]
        readfile_name = "traj-classification-features_"+scenario_name+".txt"
        record_raw = pd.read_csv(readfile_name, sep= ",", header=0) 
        print_summary = scenario_name
        #
        x_all  = record_raw[variable_x] # all
        x_base = [x_all]
        #
        n_classifications = 3
        print_count = scenario_name+"\tcount"
        print_ratio = "\tratio_sub%"
        print_avg = "\t"+variable_x+"_avg"
        print_std = "\t"+variable_x+"_std"
        for j in range(0, len(x_base)):
            x_subtotal     = x_base[j]
            x_ballistic    = x_base[j][(record_raw['motion_type']=="B")] # ballistic motion
            x_confined     = x_base[j][(record_raw['motion_type']=="C")] # confined motion
            x_subballistic = x_base[j][(record_raw['motion_type']=="S")] # sub-ballistic motion
            #
            count_subtotal     = len(x_subtotal)
            count_ballistic    = len(x_ballistic)
            count_confined     = len(x_confined)
            count_subballistic = len(x_subballistic)
            #
            avg_subtotal     = np.mean(x_subtotal)
            avg_ballistic    = np.mean(x_ballistic)
            avg_confined     = np.mean(x_confined)
            avg_subballistic = np.mean(x_subballistic)
            #
            std_subtotal     = np.std(x_subtotal)
            std_ballistic    = np.std(x_ballistic)
            std_confined     = np.std(x_confined)            
            std_subballistic = np.std(x_subballistic)   
            #
            print_count += "\t"+str(count_subtotal)+"\t"+str(count_ballistic)+"\t"+str(count_confined)+"\t"+str(count_subballistic)
            #
            print_ratio += "\t"
            print_ratio += "\t{:.2f}".format(count_ballistic/count_subtotal*100)
            print_ratio += "\t{:.2f}".format(count_confined/count_subtotal*100)
            print_ratio += "\t{:.2f}".format(count_subballistic/count_subtotal*100)  
            #
            print_avg += "\t{:.2f}".format(avg_subtotal)
            print_avg += "\t{:.2f}".format(avg_ballistic)
            print_avg += "\t{:.2f}".format(avg_confined)
            print_avg += "\t{:.2f}".format(avg_subballistic) 
            #
            print_std += "\t{:.2f}".format(std_subtotal)
            print_std += "\t{:.2f}".format(std_ballistic)
            print_std += "\t{:.2f}".format(std_confined)
            print_std += "\t{:.2f}".format(std_subballistic)             
        #
        print(print_count)
        print(print_ratio)
        print(print_avg)
        print(print_std)

In [ ]:
def show_classification_summary2(scenario_name_set, variable_x):
    #
    print_header = "scenario\tcategory\tall\t_incompleted\t_complete\tballistic\t_incompleted\t_complete\tconfined\t_incompleted\t_complete\tsubballistic\t_incompleted\t_complete"
    # print_header += "\t_parallel\tparallel_ballistic\tparallel_confined\tparallel_subballistic"
    # print_header += "\t_head-on\thead-on_ballistic\thead-on_confined\thead-on_subballistic"
    print(print_header)
    #     
    for row_id in range(0, len(scenario_name_set)):
        scenario_name = scenario_name_set[row_id]
        readfile_name = "traj-classification-features_"+scenario_name+".txt"
        record_raw = pd.read_csv(readfile_name, sep= ",", header=0) 
        print_summary = scenario_name
        #
        x_all  = record_raw[variable_x] # all
        x_base = [x_all]
        #
        n_classifications = 3
        print_count = scenario_name+"\tcount"
        print_ratio = "\tratio_sub%"
        print_avg = "\t"+variable_x+"_avg"
        print_std = "\t"+variable_x+"_std"       
        #
        for i in range(0, len(x_base)):
            for j in range(0, n_classifications+1):
                # j = 0: subtotal 
                # j = 1: ballistic
                # j = 2: confined
                # j = 3: subballistic
                x_j = []
                if (j == 0):
                    x_j = x_base[i]
                elif (j == 1):
                    x_j = x_base[i][(record_raw['motion_type']=="B")] # ballistic motion
                elif (j == 2):
                    x_j = x_base[i][(record_raw['motion_type']=="C")] # confined motion
                elif (j == 3):
                    x_j = x_base[i][(record_raw['motion_type']=="S")] # sub-ballistic motion
                else:
                    x_j = []
                count_subtotal = len(x_base[i])
                print_count += "\t"+str(len(x_j))
                if (j == 0):
                    print_ratio += "\t"
                else:
                    # print_ratio += "\t{:.2f}".format(count_ballistic/count_subtotal*100)
                    print_ratio += "\t{:.2f}".format(len(x_j)/count_subtotal*100)
                print_avg += "\t{:.2f}".format(np.mean(x_j))
                print_std += "\t{:.2f}".format(np.std(x_j))
                #
                x_k = []
                for k in range(0, 2):
                    # k = 0: contact trajectory record is incomplete (exiting_cricle = 0)
                    # k = 1: contact trajectory record is complete   (exiting_cricle = 1)
                    if (k == 0):
                        x_k = x_j[(record_raw['exiting_cricle']==0)]
                    elif (k == 1):
                        x_k = x_j[(record_raw['exiting_cricle']==1)]                    
                    else:
                        x_k = []
                    #
                    print_count += "\t"+str(len(x_k))
                    if (j == 0):
                        # print_ratio += "\t"
                        print_ratio += "\t{:.2f}".format(len(x_k)/len(x_j)*100)   
                    else:
                        # print_ratio += "\t{:.2f}".format(count_ballistic/count_subtotal*100)                    
                        print_ratio += "\t{:.2f}".format(len(x_k)/len(x_j)*100)                    
                    print_avg += "\t{:.2f}".format(np.mean(x_k))
                    print_std += "\t{:.2f}".format(np.std(x_k))                    
                #
        #
        print(print_count)
        print(print_ratio)
        print(print_avg)
        print(print_std)

In [ ]:
def show_classification_confined(scenario_name_set):
    #
    print_header = "confined motion: frequency and ratio\n\n"
    print_header += "scenario\tcontact_type"
    print_header += "\tcategory\tsubtotal\t_%\tconfined\t_%"
    print_header += "\tboth\t%_out_of_incompleted\t%_out_of_confined"
    # print_header += "\t_parallel\tparallel_ballistic\tparallel_confined\tparallel_subballistic"
    # print_header += "\t_head-on\thead-on_ballistic\thead-on_confined\thead-on_subballistic"
    print(print_header)
    #     
    for row_id in range(0, len(scenario_name_set)):
        scenario_name = scenario_name_set[row_id]
        readfile_name = "traj-classification-features_"+scenario_name+".txt"
        record_raw = pd.read_csv(readfile_name, sep= ",", header=0) 
        print_summary = scenario_name
        #
        variable_x = "tc_data"
        x_all  = record_raw[variable_x] # all
        x_base = [x_all]
        #
        print_summary = scenario_name+"\tparallel"
        #
        for i in range(0, len(x_base)):
            x_incomplete = x_base[i][(record_raw['exiting_cricle']==0)]   # incomplete contact interaction trajectory
            x_confined   = x_base[i][(record_raw['motion_type']=="C")]    # confined motion
            x_both       = x_incomplete[(record_raw['motion_type']=="C")] # confined motion having incomplete trajectory
            #
            count_subtotal   = len(x_base[i])
            count_incomplete = len(x_incomplete)
            count_confined   = len(x_confined)
            count_both       = len(x_both)
            #
            print_summary += "\t"+str(count_subtotal)
            print_summary += "\t"+str(count_incomplete)+"\t{:.2f}".format(count_incomplete/count_subtotal*100)
            print_summary += "\t"+str(count_confined)+"\t{:.2f}".format(count_confined/count_subtotal*100)
            print_summary += "\t"+str(count_both)
            print_summary += "\t{:.2f}".format(count_both/count_incomplete*100)+"\t{:.2f}".format(count_both/count_confined*100)
            #
            print(print_summary)

In [ ]:
def show_classification_tc(scenario_name_set):
    #
    print_header = "confined motion: frequency and ratio\n\n"
    print_header += "scenario\tcontact_type"
    print_header += "\tcategory\tsubtotal\tcompleted\t_%\tballistic\t_%"
    print_header += "\tboth\t%_out_of_completed\t%_out_of_ballistic"
    # print_header += "\t_parallel\tparallel_ballistic\tparallel_confined\tparallel_subballistic"
    # print_header += "\t_head-on\thead-on_ballistic\thead-on_confined\thead-on_subballistic"
    print(print_header)
    #     
    for row_id in range(0, len(scenario_name_set)):
        scenario_name = scenario_name_set[row_id]
        readfile_name = "traj-classification-features_"+scenario_name+".txt"
        record_raw = pd.read_csv(readfile_name, sep= ",", header=0) 
        print_summary = scenario_name
        #
        variable_x = "tc_data"
        x_all  = record_raw[variable_x] # all
        x_base = [x_all]
        #
        print_summary = scenario_name+"\tparallel\t"
        print_avg = "\t\t"+variable_x+"_avg"
        #
        for i in range(0, len(x_base)):
            x_complete     = x_base[i][(record_raw['exiting_cricle']==1)]   # complete contact interaction trajectory
            x_ballistic    = x_base[i][(record_raw['motion_type']=="B")]    # ballistic motion
            x_subballistic = x_base[i][(record_raw['motion_type']=="S")]    # sub-ballistic motion
            x_confined     = x_base[i][(record_raw['motion_type']=="C")]    # confined motion
            x_both         = x_complete[(record_raw['motion_type']=="B")]   # ballistic motion having complete trajectory
            #
            count_subtotal   = len(x_base[i])
            count_complete   = len(x_complete)
            count_ballistic  = len(x_ballistic)
            count_both       = len(x_both)
            #
            print_summary += "\t"+str(count_subtotal)
            print_summary += "\t"+str(count_complete)+"\t{:.2f}".format(count_complete/count_subtotal*100)
            print_summary += "\t"+str(count_ballistic)+"\t{:.2f}".format(count_ballistic/count_subtotal*100)
            print_summary += "\t"+str(count_both)
            print_summary += "\t{:.2f}".format(count_both/count_complete*100)+"\t{:.2f}".format(count_both/count_ballistic*100)
            #
            print_avg += "\t{:.2f}".format(np.mean(x_base[i]))
            print_avg += "\t{:.2f}".format(np.mean(x_complete))+"\t"
            print_avg += "\t{:.2f}".format(np.mean(x_ballistic))+"\t"
            print_avg += "\t{:.2f}".format(np.mean(x_both))+"\t"+"\t"
            #
            print(print_summary)
            print(print_avg)

In [ ]:
def generate_histograms_classification_completed(scenario_name_set, fps, variable_x, label_x, x_binwidth, x_min_i="", x_max_i="", xtick_space=""):
    #
    # prepare subplots
    #
    n_types = 1
    n_classifications = 3
    n_scenarios = len(scenario_name_set)
    #
    n_columns = n_classifications+1
    n_rows    = n_types*n_scenarios
    #
    subplot_x = 4.0
    subplot_y = 3.0
    figsize_x = subplot_x*n_columns
    figsize_y = subplot_y*n_rows
    fig = plt.figure(figsize=(figsize_x, figsize_y))       
    #
    check_x_min = False
    check_x_max = False
    if not x_min_i:
        check_x_min = True
    if not x_max_i:
        check_x_max = True
    #
    # for i in range (0, len(scenario_name_set)):
    for i in range(0, len(scenario_name_set)):
        scenario_name = scenario_name_set[i]
        row_id = i
        dt = 1.0/fps
        # scenario_name = scenario_name_set[i]
        readfile_name = "traj-classification-features_"+scenario_name+".txt"
        record_raw = pd.read_csv(readfile_name, sep= ",", header=0) 
        #
        x_0 = record_raw[variable_x] # all
        x_0 = x_0[(record_raw['exiting_cricle']==1)] # complete trajectories only
        subplot_title_0 = scenario_name
        #
        x_1 = x_0[(record_raw['motion_type']=="B")] # ballistic motion
        x_2 = x_0[(record_raw['motion_type']=="C")] # confined motion
        x_3 = x_0[(record_raw['motion_type']=="S")] # sub-ballistic motion
        x = [x_0, x_1, x_2, x_3]
        print(scenario_name, len(x_0), len(x_1), len(x_2), len(x_3))
        #
        subplot_title = []
        subplot_title.append(subplot_title_0)
        subplot_title.append("parallel :: ballistic")
        subplot_title.append("parallel :: confined")
        subplot_title.append("parallel :: sub-ballistic")
        #
        if (check_x_min == True):
            x_min = min(min(x[0]), 0.0)  
        else: 
            x_min = x_min_i
        if (check_x_max == True):
            x_max = max(x[0])  
        else: 
            x_max = x_max_i            
        #
        n_xticks = 5
        #
        for j in range(0, n_columns):
            if (len(x[j]) > 0):                
                ax = fig.add_subplot(n_rows, n_columns, (n_columns*row_id+j+1))
                bin_list = np.arange(x_min, x_max*1.05, x_binwidth) 
                # counts = plt.hist(x[j], bins=bin_list, alpha=0.5, weights=np.ones_like(x[j])/len(x[j])) # alpha controls transparency
                counts = plt.hist(x[j], bins=bin_list, alpha=0.5) # alpha controls transparency
                #
                plt.xlim(x_min, x_max)
                if not xtick_space:
                    ax.xaxis.set_major_locator(plt.MaxNLocator(n_xticks))
                else:
                    plt.xticks(np.arange(x_min, x_max*1.05, xtick_space))       
                plt.tick_params(labelsize=12)
                if (row_id == (n_rows-1)):
                    plt.xlabel(label_x, fontsize=12)
                if (j == 0):
                    plt.ylabel('frequency', fontsize=12) 
                plt.title(subplot_title[j], fontsize=16)               
    # set the spacing between subplots
    plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.4, hspace=0.4)    
    plt.show()        

In [ ]:
def count_parallel(scenario_name_set):
    #
    print_header = "scenario\t_all\tall_ballistic\tall_confined"
    print(print_header)
    variable_x = "tc_data"
    #     
    for row_id in range(0, len(scenario_name_set)):
        scenario_name = scenario_name_set[row_id]
        readfile_name = "traj-classification-features_"+scenario_name+".txt"
        record_raw = pd.read_csv(readfile_name, sep= ",", header=0) 
        print_summary = scenario_name
        #
        x_all       = record_raw[variable_x] # all
        x_ballistic = record_raw[variable_x][(record_raw['motion_type']=="B")] # ballistic motion
        x_confined = record_raw[variable_x][(record_raw['motion_type']=="C")] # confined motion
        #
        print_count = scenario_name+"\t"+str(len(x_all))+"\t"+str(len(x_ballistic))+"\t"+str(len(x_confined))
        print(print_count)

In [ ]:
def check_count_tc(scenario_name_set):
    #
    print_header = "scenario\t_all\tall_parallel\tall_ballistic\tall_confined\ttc_confined\ttc_all"
    print(print_header)
    variable_x = "tc_data"
    #     
    for row_id in range(0, len(scenario_name_set)):
        scenario_name = scenario_name_set[row_id]
        readfile_name = "traj-classification-features_"+scenario_name+".txt"
        record_raw = pd.read_csv(readfile_name, sep= ",", header=0) 
        print_summary = scenario_name
        #
        x_all       = record_raw[variable_x] # all
        x_ballistic = record_raw[variable_x][(record_raw['motion_type']=="B")] # ballistic motion
        x_confined  = record_raw[variable_x][(record_raw['motion_type']=="C")] # confined motion
        tc_confined = np.mean(x_confined)
        tc_all      = np.mean(x_all)
        #
        print_count = scenario_name+"\t"+str(len(x_all))+"\t"+str(len(x_all))+"\t"+str(len(x_ballistic))
        print_count += "\t"+str(len(x_confined))+"\t{:.1f}".format(tc_confined)+"\t{:.1f}".format(tc_all)
        print(print_count)                

In [ ]:
def generate_annotated_heatmaps_individuals(scenario_name_set):
    #
    # creating annotated heatmaps
    # see tips: 
    # https://matplotlib.org/stable/gallery/images_contours_and_fields/image_annotated_heatmap.html
    # https://matplotlib.org/stable/gallery/color/colorbar_basics.html
    # 
    variable_x = "tc_data"
    for k in range (0, len(scenario_name_set)):
        # x-axis: motion type -- ballistic, confined, sub-ballistic
        # y-axis: contact interaction types -- parallel, head-on, crossing        
        #
        scenario_name = scenario_name_set[k]
        readfile_name = "traj-classification-features_"+scenario_name+".txt"
        record_raw = pd.read_csv(readfile_name, sep= ",", header=0) 
        #
        interaction_types = ["parallel"]
        motion_types = ["ballistic", "confined", "sub-ballistic"]
        motion_values = ["B", "C", "S"]
        #        
        count_classification = []
        for i in range(0, len(interaction_types)):    
            count_i = []
            for j in range(0, len(motion_types)):
                count_ij = len(record_raw[variable_x][record_raw['motion_type']==motion_values[j]])
                count_i.append(count_ij)
                # count_ij = len(record_raw[variable_x])
                ## print(scenario_name_set[k], interaction_types[i], motion_types[j], count_ij)
            ## print(scenario_name_set[k], count_i)
            count_classification.append(count_i)
        #
        count_classification = np.array(count_classification)
        ratio_classification = count_classification/np.sum(count_classification)
        threshold = count_classification.max()*0.75
        #
        figsize_x = 5.5
        figsize_y = 2.0
        #fig, ax = plt.subplots(figsize=(figsize_x, figsize_y))
        fig, ax = plt.subplots()
        im = ax.imshow(ratio_classification, cmap = "YlGn", interpolation='none', vmin=0, vmax=1)

        # Show all ticks and label them with the respective list entries
        ax.set_xticks(np.arange(len(motion_types)))
        ax.set_yticks(np.arange(len(interaction_types)))

        # Rotate the tick labels and set their alignment.
        plt.setp(ax, xticklabels = motion_types, yticklabels = interaction_types)
        plt.xticks(fontsize=18, rotation=45)
        plt.yticks(fontsize=18)
        # Loop over data dimensions and create text annotations
        for i in range(0, len(motion_types)):
            for j in range(0, len(interaction_types)):    
                color_ij = "black"
                if (count_classification[j, i] > threshold): color_ij = "white"
                text = ax.text(i, j, count_classification[j, i], ha="center", va="center", size=20, color=color_ij)            
        #
        ax.set_title(scenario_name_set[k], fontsize = 24)
        fig.tight_layout()
        plt.show()        

In [ ]:
def generate_annotated_heatmaps_avg(scenario_name_set, variable_x):
    #
    # creating annotated heatmaps
    # see tips: 
    # https://matplotlib.org/stable/gallery/images_contours_and_fields/image_annotated_heatmap.html
    # https://matplotlib.org/stable/gallery/color/colorbar_basics.html
    # 
    for k in range (0, len(scenario_name_set)):
        # x-axis: motion type -- ballistic, confined, sub-ballistic
        # y-axis: contact interaction types -- parallel, head-on, crossing        
        #
        scenario_name = scenario_name_set[k]
        readfile_name = "traj-classification-features_"+scenario_name+".txt"
        record_raw = pd.read_csv(readfile_name, sep= ",", header=0) 
        #
        interaction_types = ["parallel"]
        motion_types = ["ballistic", "confined", "sub-ballistic"]
        motion_values = ["B", "C", "S"]
        #        
        avg_classification = []
        for i in range(0, len(interaction_types)):    
            avg_i = []
            for j in range(0, len(motion_types)):
                x_ij = record_raw[variable_x][record_raw['motion_type']==motion_values[j]]
                avg_ij = np.mean(x_ij)
                avg_i.append(avg_ij)
            avg_classification.append(avg_i)
        #
        avg_classification = np.array(avg_classification)
        threshold = np.nanmax(avg_classification)*0.7 
        #
        figsize_x = 5.5
        figsize_y = 2.0
        fig, ax = plt.subplots()
        # fig, ax = plt.subplots(figsize=(figsize_x, figsize_y))
        # im = ax.imshow(avg_classification, cmap = "YlGn", interpolation='none', vmin=0, vmax=1)
        im = ax.imshow(avg_classification, cmap = "YlGn", interpolation='none', vmin=0)

        # Show all ticks and label them with the respective list entries
        ax.set_xticks(np.arange(len(motion_types)))
        ax.set_yticks(np.arange(len(interaction_types)))

        # Rotate the tick labels and set their alignment.
        plt.setp(ax, xticklabels = motion_types, yticklabels = interaction_types)
        plt.xticks(fontsize=18, rotation=45)
        plt.yticks(fontsize=18)
        
        # Loop over data dimensions and create text annotations
        
        for i in range(0, len(motion_types)):
            for j in range(0, len(interaction_types)):    
                color_ij = "black"
                if (avg_classification[j, i] > threshold): color_ij = "white"
                text = ax.text(i, j, "{:.1f}".format(avg_classification[j, i]), ha="center", va="center", size=20, color=color_ij, fontweight="bold")    
        """
        for i in range(0, len(motion_types)):
            for j in range(0, len(interaction_types)):    
                text = ax.text(i, j, "{:.1f}".format(avg_classification[j, i]), ha="center", va="center", size=16)     
        """
        #
        ax.set_title(scenario_name_set[k], fontsize = 24)
        fig.tight_layout()
        plt.show()        

In [ ]:
variable_x = "n_datapoints"
label_x = "No. position data points"
x_binwidth = 10
generate_histograms(scenario_name_set, fps_values[0], variable_x, label_x, x_binwidth)

In [ ]:
variable_x = "tc_data"
label_x = "contact duration (s)"
x_binwidth = 0.5
# generate_histograms_relative(scenario_name_set, fps_values[0], variable_x, label_x, x_binwidth)
x_min = 0.0
# x_max = 20.0
# generate_histograms_relative(scenario_name_set, fps_values[0], variable_x, label_x, x_binwidth, x_min, x_max)
generate_histograms(scenario_name_set, fps_values[0], variable_x, label_x, x_binwidth, x_min)

In [ ]:
variable_x = "turning_angle_entropy"
label_x = "turning angle entropy"
x_binwidth = 0.05
x_min = 0.0
x_max = 1.0
generate_histograms(scenario_name_set, fps_values[0], variable_x, label_x, x_binwidth, x_min, x_max)

In [ ]:
variable_x = "efficiency"
label_x = "efficiency"
x_binwidth = 0.05
x_min = 0.0
x_max = 1.0
generate_histograms(scenario_name_set, fps_values[0], variable_x, label_x, x_binwidth, x_min, x_max)

In [ ]:
variable_x = "tc_data"
label_x = "contact duration (s)"
x_binwidth = 0.5
x_min = 0.0
#
generate_histograms_classification(scenario_name_set, fps_values[0], variable_x, label_x, x_binwidth, x_min)

In [ ]:
variable_x = "tc_data"
label_x = "contact duration (s)"
x_binwidth = 0.5
x_min = 0.0
#
generate_histograms_classification(["uni-05"], fps_values[0], variable_x, label_x, x_binwidth, x_min)

In [ ]:
#
variable_x = "tc_data"
show_classification_count(scenario_name_set, variable_x)

In [ ]:
variable_x = "tc_data"
show_classification_summary1(scenario_name_set, variable_x)

In [ ]:
variable_x = "tc_data"
show_classification_summary2(scenario_name_set, variable_x)

In [ ]:
show_classification_confined(scenario_name_set)

In [ ]:
show_classification_tc(scenario_name_set)

In [ ]:
variable_x = "tc_data"
label_x = "contact duration (s)"
x_binwidth = 0.5
x_min = 0.0
x_max = ""
xtick_space = ""
#
generate_histograms_classification_completed(scenario_name_set, fps, variable_x, label_x, x_binwidth, x_min, x_max, xtick_space)

In [ ]:
variable_x = "v_avg"
label_x = "average contact speed (m/s)"
x_binwidth = 0.1
x_min = 0.0
x_max = ""
xtick_space = ""
#
generate_histograms_classification(scenario_name_set, fps, variable_x, label_x, x_binwidth, x_min, x_max, xtick_space)

In [ ]:
count_parallel(scenario_name_set)

In [ ]:
check_count_tc(scenario_name_set)

In [ ]:
generate_annotated_heatmaps_individuals(scenario_name_set)

In [ ]:
generate_annotated_heatmaps_avg(scenario_name_set, "tc_data")